In [ ]:
import scipy.io
import scipy
import scipy.spatial
import scipy.ndimage
import numpy as np
import os
import torch
from torchvision import datasets, transforms, models
from torch.utils.tensorboard import SummaryWriter
import gc

import matplotlib.pyplot as plt
from torch.utils.data import Dataset
import zipfile
import random


import cv2
from PIL import Image,ImageFilter,ImageDraw
from PIL import ImageStat


from model import CSRNet
from dataset import listDataset

from tqdm.notebook import tqdm

In [ ]:
# extract_data(path2zip='ShangaiTech.zip', dir_name='ShangaiTech')
rootShangai_im = 'ShangaiTech/ShanghaiTech/part_A/train_data/images/'
rootShangai_gt = 'ShangaiTech/ShanghaiTech/part_A/train_data/ground-truth/'

In [ ]:
def mat2numpy(path2jpg=str):

    path2mat = path2jpg.replace('images', 'ground-truth').replace('IMG', 'GT_IMG').replace('jpg', 'mat')
    image = np.asarray(Image.open(path2jpg))

    print(path2jpg, path2mat)

    ground_truth = scipy.io.loadmat(path2mat)
    ground_truth = ground_truth['image_info'][0][0][0][0][0].astype(int)

    return image, ground_truth

In [ ]:
def density_map(ground_truth=np.ndarray, img_shape=np.ndarray, k = 4, beta = 0.3):

    if len(img_shape) == 3:
        im_size = img_shape[:-1]
    else: im_size = img_shape
        
    gt = ground_truth.copy()

    # head_loc = np.zeros(im_size, dtype=np.float32)
    # head_loc[gt[:,1], gt[:,0]] = 1.

    density = np.zeros(im_size, dtype=np.float32)

    counts = len(gt)
    if counts == 0.:
        return density

    leafsize = 2048
    # build kdtree
    tree = scipy.spatial.KDTree(gt.copy(), leafsize=leafsize)
    # query kdtree
    distances, index = tree.query(gt, k=k)

    for i, pt in enumerate(gt):
        pt2d = np.zeros(im_size, dtype=np.float32)
        pt2d[pt[1], pt[0]] = 1.

        if counts >= k:
            near_pt_coordinate = gt[index.astype(int)[i]]
            near_pt_coordinate[:, 0] = near_pt_coordinate[:, 0] - pt[0]
            near_pt_coordinate[:, 1] = near_pt_coordinate[:, 1] - pt[1]
        
            sigma_x = np.sum(np.abs(near_pt_coordinate[:, 0]))/(k-1)
            sigma_y = np.sum(np.abs(near_pt_coordinate[:, 1]))/(k-1)
            sigma = [sigma_x*beta, sigma_y*beta]
            # sigma = distances[i, 1:].mean()*beta
            # sigma = (distances[i][1]+distances[i][2]+distances[i][3])*0.1

        else:
            sigma = np.average(np.array(im_size))/2./2. #case: 1 point
            
        density += scipy.ndimage.gaussian_filter(pt2d, sigma, mode='constant')
    
    return density

In [ ]:
rootShangai_train_newdensity = 'ShangaiTech_density/ShangaiTech_newdensity/A/train_data/'
rootShangai_test_newdensity = 'ShangaiTech_density/ShangaiTech_newdensity/A/test_data/'

In [ ]:
for im_name in os.listdir(rootShangai_im)[60:]:
    img , gt = mat2numpy(path2jpg=rootShangai_im+im_name)
    den = density_map(gt, img.shape)

    
    np.save(file=rootShangai_train_newdensity+im_name.replace('IMG', 'DEN').replace('jpg', 'npy'), arr=den)

In [ ]:
rootShangai_im = 'ShangaiTech/ShanghaiTech/part_A/test_data/images/'
for im_name in os.listdir(rootShangai_im):
    img , gt = mat2numpy(path2jpg=rootShangai_im+im_name)
    den = density_map(gt, img.shape)
    
    np.save(file=rootShangai_test_newdensity+im_name.replace('IMG', 'DEN').replace('jpg', 'npy'), arr=den)